Анализатор 1inch fusion and OpenOcean. Наброски кода для детального обсуждения первой версии сервиса.

Стратегия анализатор рейтов Вариант1 (1inch fusion Swap -> OpenOcean Swap)  
Предполагается, что точкой входа для анализа рейтов будет получение активных ордеров на 1inch fusion,
все полученные ордера сохраняются в БД со статусом "pending".
На данном этапе известно, что все fusion ордера - это Dutch auction filling rate. Цена будет падать до минимальной, и в итоге ордер будет исполнен резолвером или просрочен/отменен/и т.п. Есть несколько статусов, которые требуют детального изучения
(реакции), но на данном этапе будем работать со статусом "pending". Если статус поменялся, то ордер выпадает из дальнейшего анализа.

Таким образом основная механика стратегии следующая:  

   1. запрашиваем новые ордера на 1inch fusion Swap, добавляем в БД  
   2. сравниваем рейты с OpenOcean Swap и помечаем потенциально выгодные для сделки  
   3. проверим статусы оредра на 1inch fusion, все которые НЕ "pending" - меняем статус и исключаем из мониторинга (или удаляем из БД)  
    
далее пункт 1-2-3 по циклу (в будущем это могут быть разные задачи (процессы))

In [38]:
# 1. запрашиваем новые ордера на 1inch fusion Swap, добавляем в БД

import requests
import pprint

NETWORK = 1
URL_API_1INCH = f'https://fusion.1inch.io/orders/v1.0/{NETWORK}'
URL_ORDER_ACTIVE = f'{URL_API_1INCH}/order/active/'
orders = requests.get(URL_ORDER_ACTIVE, params={'page': 1, 'limit': 1}).json()
pprint.pprint(orders['meta'])
pprint.pprint(orders['items'])

URL_ORDER_STATUS = f'{URL_API_1INCH}/order/status/'

{'currentPage': 1, 'itemsPerPage': 1, 'totalItems': 3, 'totalPages': 3}
[{'auctionEndDate': '2023-04-05T10:15:23.000Z',
  'auctionStartDate': '2023-04-05T10:12:23.000Z',
  'deadline': '2023-04-05T10:15:35.000Z',
  'order': {'allowedSender': '0xa88800cd213da5ae406ce248380802bd53b47647',
            'interactions': '0x2cc2878d0000642d4a4700000000000064b0e2ab2cfe00362491c7866a19b546a98e9ae30000000084d99aa569d93a9ca187d83734c8c4a519c4e9b100000000c6c7565644ea1893ad29182f7b6961aab7edfed000000000d1742b3c4fbb096990c8950fa635aec75b30781a000000007636a5bfd763cefec2da9858c459f2a9b0fe8a6c00000000bd4dbe0cb9136ffb4955ede88ebd5e92222ad09a0000000069313aec23db7e4e8788b942850202bcb603873400000000cfa62f77920d6383be12c91c71bd403599e1116f00000000ee230dd7519bc5d0c9899e8704ffdc80560e8509000000009108813f22637385228a1c621c1904bbbc50dc25642d49ff48',
            'maker': '0x64b0e2ab2cfe00362491c7866a19b546a98e9ae3',
            'makerAsset': '0xfcf8eda095e37a41e002e266daad7efc1579bc0a',
            'makingAmount'

детальное описание:
https://github.com/1inch/fusion-sdk/tree/main/src/auction-calculator

In [39]:
# добавление записей в БД, записываем в записи, так как рейт будет уменьшаться, потенциально все записи могут быть выгодные для сделки 
def save_to_db(orders):
    # реализовать функцию на данном этапе нет необходимости
    pass
save_to_db(orders)

In [41]:
# 2. сравниваем рейты с OpenOcean Swap и помечаем потенциально выгодные для сделки
def get_orders_from_db():
    # реализовать функцию на данном этапе нет необходимости
    return orders

orders_from_db = get_orders_from_db()
order = orders_from_db['items'][0]['order']

in_token_address = order['makerAsset']
out_token_address = order['takerAsset']
amount = order['makingAmount']
SLIPPAGE = 1
deciamls = 18 # в зависимости от адреса токена

URL_API_OPENOCEAN_QUOTE = 'https://open-api.openocean.finance/v3/eth/quote'
quote = requests.get(URL_API_OPENOCEAN_QUOTE, params={
    'inTokenAddress': in_token_address, 
    'outTokenAddress': out_token_address,
    'amount': int(amount) / 10**deciamls,
    'gasPrice': 5,
    'slippage': SLIPPAGE,
}).json()
# https://docs.openocean.finance/dev/openocean-api-3.0/quick-start
pprint.pprint(quote)

{'code': 200,
 'data': {'dexes': [{'dexCode': 'UniswapV2',
                     'dexIndex': 0,
                     'swapAmount': '1011649763'},
                    {'dexCode': 'SushiSwap', 'dexIndex': 1, 'swapAmount': '0'},
                    {'dexCode': 'Aave', 'dexIndex': 3, 'swapAmount': '0'},
                    {'dexCode': 'AaveV2', 'dexIndex': 4, 'swapAmount': '0'},
                    {'dexCode': 'UniswapV3',
                     'dexIndex': 5,
                     'swapAmount': '996652279'},
                    {'dexCode': 'Curve', 'dexIndex': 6, 'swapAmount': '0'},
                    {'dexCode': 'BalancerV2', 'dexIndex': 7, 'swapAmount': '0'},
                    {'dexCode': 'KyberSwap', 'dexIndex': 8, 'swapAmount': '0'},
                    {'dexCode': 'ShibaSwap', 'dexIndex': 9, 'swapAmount': '0'},
                    {'dexCode': 'DODOv2', 'dexIndex': 10, 'swapAmount': '0'},
                    {'dexCode': '1Inch', 'dexIndex': 11, 'swapAmount': '0'},
                    {

In [ ]:
def good_deal(order, percent):
    pass